In [1]:
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
model_encoder = SentenceTransformer("jhgan/ko-sroberta-multitask")

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model_encoder.encode(sentences)

print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.37510458 -0.77338415  0.592771   ...  0.5792353   0.32683477
  -0.6508964 ]
 [-0.09361707 -0.1819153  -0.19230859 ... -0.03165823  0.30412504
  -0.26793605]]


In [6]:
df = pd.read_csv('https://github.com/kairess/mental-health-chatbot/raw/master/wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


##비어있는 칸을 없애주기 -> 데이터 전처리

In [9]:
df = df.drop(columns=['Unnamed: 3'])
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [10]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

##벡터화 된 모습 확인

In [12]:
model_encoder.encode(df.loc[0, '유저'])

array([-4.80606556e-01, -2.94869244e-01,  4.37900126e-01, -6.40137792e-01,
        3.28673832e-02, -3.42647463e-01, -5.47483228e-02,  1.73053443e-02,
       -4.08220530e-01, -5.06034136e-01, -1.68733597e-01, -3.98677140e-01,
       -1.24776483e-01, -9.71537307e-02, -1.65286541e-01,  5.72599703e-03,
        6.13690913e-02, -1.91312149e-01,  2.53917158e-01, -5.85019529e-01,
       -2.84425765e-01, -2.32034937e-01, -3.27080697e-01,  6.72975555e-02,
       -1.64721696e-05, -4.72336322e-01, -3.60021859e-01,  2.91879892e-01,
       -6.63861275e-01, -3.10574561e-01,  5.79524994e-01, -3.11722964e-01,
        1.47699434e-02, -2.12172642e-01,  2.22058162e-01, -1.73829034e-01,
       -3.78458351e-01, -4.20398235e-01, -2.38218814e-01,  6.38703629e-02,
       -1.15304172e-01, -2.44564027e-01, -5.00228465e-01,  1.68355554e-01,
       -6.58360064e-01, -8.91941905e-01, -6.26957119e-01, -3.21965367e-01,
       -7.05358505e-01,  3.71447653e-01, -5.45803249e-01,  7.76298046e-02,
        1.09864749e-01,  

##모든 데이터 벡터화 한 변수인 embedding을 추가

In [13]:
df['embedding'] = pd.Series([[]] * len(df))
df['embedding'] = df['유저'].map(lambda x: list(model_encoder.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060656, -0.29486924, 0.43790013, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506273, 0.29490337, -0.67394..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6651999, -0.08126834, 1.0945568, 0.1057920..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679059, 0.46520725, 0.528507, -0.5076046,..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277755, -0.37413907, 0.04053206, -0.8624..."


In [14]:
df.to_csv('wellness_dataset.csv')

##챗봇 테스트하기

In [17]:
text = '과제도 너무 많고 힘들어'

embedding = model_encoder.encode(text)

##텍스트와 가장 유사한 문장을 찾기
##cosine_similarity로 각각 데이터 문장과의 거리를 계산해서 유사한 문장 찾기

In [19]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance,similarity
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060656, -0.29486924, 0.43790013, -0.6401...",0.176240,0.176240
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506273, 0.29490337, -0.67394...",0.249185,0.249185
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6651999, -0.08126834, 1.0945568, 0.1057920...",0.264345,0.264345
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679059, 0.46520725, 0.528507, -0.5076046,...",0.330203,0.330203
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277755, -0.37413907, 0.04053206, -0.8624...",0.238737,0.238737


##비슷한 내용의 데이터에서 추출한 내용들을 보기

In [25]:
answer = df.loc[df['similarity'].idxmax()]

print('구분:', answer['구분'])
print('유사한 질문:', answer['유저'])
print('챗봇 답변:', answer['챗봇'])
print('유사도:', answer['similarity'])

구분: 감정/힘듦/스트레스
유사한 질문: 스트레스가 너무 심하네요.
챗봇 답변: 정말 힘들고 스트레스 받으시겠어요. 따뜻한 물에 샤워를 추천해드릴게요.
유사도: 0.6248472929000854


In [26]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s et

In [28]:
import gradio as gr

def greet(user, history=[]):
    embedding = model_encoder.encode(user)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    history.append([user, answer['챗봇']])

    return history, history

demo = gr.Interface(fn=greet, inputs=["text", "state"], outputs=["chatbot", "state"])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://121714e1b0687c602b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://121714e1b0687c602b.gradio.live
